In [2]:
import wget
import requests
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from Andrew import sandwich
import numpy as np
import os
import time
import re
import glob
from pathlib import Path
import re

In [3]:
df=pd.read_csv("International Travel Jan 10.csv")
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Page #,Departure,Return,Year,Country
0,0,0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE
1,1,1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE
2,2,2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE
3,3,3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE
4,4,4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,1,"Dec. 7, 2023","Dec. 11, 2023",2023,UAE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,6195,4,"Nov. 8, 2018 - Nov. 13, 2018",Carlos Curbelo,Carlos Curbelo,R,FL-26,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France
1629,6196,5,"Nov. 8, 2018 - Nov. 13, 2018",Peter Welch,Peter Welch,D,VT-1,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France
1630,6197,6,"Nov. 7, 2018 - Nov. 12, 2018",Mike Doyle,Mike Doyle,D,PA-0,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France
1631,6198,7,"Nov. 7, 2018 - Nov. 12, 2018",Josh Gottheimer,Josh Gottheimer,D,NJ-5,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France


In [4]:
df["Raw Costs"]=""
df=df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
df


,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Page #,Departure,Return,Year,Country,Raw Costs
0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE,
1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE,
2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE,
3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE,
4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,1,"Dec. 7, 2023","Dec. 11, 2023",2023,UAE,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,"Nov. 8, 2018 - Nov. 13, 2018",Carlos Curbelo,Carlos Curbelo,R,FL-26,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France,
1629,"Nov. 8, 2018 - Nov. 13, 2018",Peter Welch,Peter Welch,D,VT-1,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France,
1630,"Nov. 7, 2018 - Nov. 12, 2018",Mike Doyle,Mike Doyle,D,PA-0,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France,
1631,"Nov. 7, 2018 - Nov. 12, 2018",Josh Gottheimer,Josh Gottheimer,D,NJ-5,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France,


In [5]:
len(df)

1633

In [6]:
#Download and scrape the pdf of a privately-sponsored congressional trip
def read_housefiles(file):
    '''
    Takes a house file(pdf) and reads the entries for traveler cost and accompanying 
    family member cost. Returns "error" if the file cannot be read.
    '''
    print(f"Reading{file}\n")
    doc=convert_from_path(file)
    costs=[]
    print(f"now scraping {file}")
    for page in doc: 
        text=pytesseract.image_to_string(page)
        lower=text.lower()
        if (seektext1 in lower) and (seektext2 in lower):
            costs=sandwich(text, start_text, end_text)
            if costs!="Error making sandwich":
                print(f"trip cost {costs}")
                return costs
                break
            else:
                print(f"unable to scrape costs in {file}")
                return costs
        else:
            pass
    if costs==[]:
        return "error"

In [ ]:
seektext1="actual amount of expenses paid on behalf of"
seektext2="sponsor post-travel disclosure form"
start_text="and description"
end_text="Family Member"

In [ ]:
def fitz_read(file):
    doc=fitz.open(file)
    costs=[]
    for page in doc:
        text=page.get_text()
        lower=text.lower()
        if(seektext1 in lower) and (seektext2 in lower):
            costs=sandwich(text, start_text,end_text)
            if costs!="Error making sandwich":
                print(f"trip cost {costs}")
                return costs
                break
            else:
                print(f"unable to scrape costs in {file}")
                return costs
        else:
            pass
    if costs==[]:
        return "error"

In [5]:
def cleancosts(string):
    '''
    Takes the string returned by the previous function. Attempts to strip out dollar signs
    and other mischellanea to add up the total cost of a trip. 
    If there's still leftover text, it returns the original string
    '''
    string=str(string)
    newstring=string.strip().replace("Traveler ","").replace("|","").replace("/","").replace("\\","")
    newstring=newstring.replace(" Accompanying ","").replace("1$","")
    newstring=newstring.replace("Accompanying ","")
    newstring=newstring.replace("$","").replace(",","").replace("\n","")
    numblist=newstring.split(" ")
    try:
        numbers=[float(x) for x in numblist]
        return sum(numbers)
    except: 
        string=string

In [6]:
df["File saved as"]=None
df

,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,File saved as
0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,,None
1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,,None
2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,,None
3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,,None
4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,,None
...,...,...,...,...,...,...,...,...,...,...,...,...
30045,"Sept. 30, 2007 - Oct. 1, 2007",Jeff Hild,Pete Stark,D,CA-13,"Baltimore, MD",National Helath Policy Forum,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,None
30046,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,None
30047,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,None
30048,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,None


In [7]:
#Where you want to download your files
downloads=glob.glob(r"../../../../../../Volumes/Andrew+Echo/Congressional Gift Travel/House Files")

In [ ]:
broken_links=[]
costs_dict=[]
no_forms=[]
for i in range(0, 1634):
    row=df.loc[i]
    raw_costs=[]
    destination=row['Country']
        #docfile=row["hrefs"]
    url=row["hrefs"]
    if destination!="USA":
        filename=f"Downloads/{i}{row['Traveler']}{row['Dates']}.pdf"
        response=requests.get(url)
        if response.status_code<400:
            print(f"now downloading information on {row['Traveler']}'s trip to {destination}' on {row['Dates']}")
            wget.download(url, out=filename)
            raw_costs=read_housefiles(filename)
            if raw_costs=="error":
                no_forms.append([i, row['Traveler'], row['Dates'],destination])
                print(f"No disclosure found in row {i}")
            else:
                print(f"{row['Traveler']}'s trip cost:\n {raw_costs}\n")
            df.loc[i,"Raw Costs"]=raw_costs
            os.remove(filename)
        else:
            print(f"failed to read {row['Traveler']} on {row['Dates']}")
            broken_links.append({"row":i, "traveler":row['Traveler'],"Date":row["Dates"],"url":url})
    else:
        pass
    time.sleep(1)
print("Scraping complete")

now downloading information on Vivian Moeglein's trip to UAE' on Dec. 6, 2023 - Dec. 12, 2023
100% [....................................................] 23790435 / 23790435ReadingDownloads/0Vivian MoegleinDec. 6, 2023 - Dec. 12, 2023.pdf

now scraping Downloads/0Vivian MoegleinDec. 6, 2023 - Dec. 12, 2023.pdf
trip cost )

$1,738*

$1,702.90 *Averaged increased

Accompanying |aFlights & local

Vivian Moeglein's trip cost:
 )

$1,738*

$1,702.90 *Averaged increased

Accompanying |aFlights & local


now downloading information on Aniela Butler's trip to UAE' on Dec. 6, 2023 - Dec. 12, 2023
100% [......................................................] 1473254 / 1473254ReadingDownloads/1Aniela ButlerDec. 6, 2023 - Dec. 12, 2023.pdf

now scraping Downloads/1Aniela ButlerDec. 6, 2023 - Dec. 12, 2023.pdf
unable to scrape costs in Downloads/1Aniela ButlerDec. 6, 2023 - Dec. 12, 2023.pdf
Aniela Butler's trip cost:
 Error making sandwich

now downloading information on Brandon Mooney's trip to U

now downloading information on Drew Lingle's trip to UAE' on Dec. 6, 2023 - Dec. 11, 2023
100% [......................................................] 2984082 / 2984082ReadingDownloads/18Drew LingleDec. 6, 2023 - Dec. 11, 2023.pdf

now scraping Downloads/18Drew LingleDec. 6, 2023 - Dec. 11, 2023.pdf
unable to scrape costs in Downloads/18Drew LingleDec. 6, 2023 - Dec. 11, 2023.pdf
Drew Lingle's trip cost:
 Error making sandwich

now downloading information on Michael Cameron's trip to Netherlands' on Oct. 30, 2023 - Nov. 3, 2023
100% [......................................................] 2612334 / 2612334ReadingDownloads/19Michael CameronOct. 30, 2023 - Nov. 3, 2023.pdf

now scraping Downloads/19Michael CameronOct. 30, 2023 - Nov. 3, 2023.pdf
trip cost )

Total Transportation
Expenses

Traveler

Accompanying

Michael Cameron's trip cost:
 )

Total Transportation
Expenses

Traveler

Accompanying


now downloading information on Tom McClintock's trip to United Kingdom' on Oct. 29, 2023

now downloading information on Monika Konrad's trip to South Korea' on Oct. 7, 2023 - Oct. 13, 2023
100% [......................................................] 2364129 / 2364129ReadingDownloads/34Monika KonradOct. 7, 2023 - Oct. 13, 2023.pdf

now scraping Downloads/34Monika KonradOct. 7, 2023 - Oct. 13, 2023.pdf
trip cost )

Traveler .

$3000 per person forounc-p, | S230PP per night — | $355pp total N/A

flight from Washington, DC - $1 .150pp total for 5

Seoul, South Korea .

nights

Accompanying

Monika Konrad's trip cost:
 )

Traveler .

$3000 per person forounc-p, | S230PP per night — | $355pp total N/A

flight from Washington, DC - $1 .150pp total for 5

Seoul, South Korea .

nights

Accompanying


now downloading information on Elizabeth Mount's trip to South Korea' on Oct. 7, 2023 - Oct. 13, 2023
100% [......................................................] 7123055 / 7123055ReadingDownloads/35Elizabeth MountOct. 7, 2023 - Oct. 13, 2023.pdf

now scraping Downloads/35Elizabeth M

100% [......................................................] 4410460 / 4410460ReadingDownloads/47Sheila Cherfilus-McCormickSept. 1, 2023 - Sept. 10, 2023.pdf

now scraping Downloads/47Sheila Cherfilus-McCormickSept. 1, 2023 - Sept. 10, 2023.pdf
trip cost )
Traveler $19,590.91 $1,513.23 $1,430.13 $4,651.21
Accompanying /$19,590.91 $1,513.23 $1,430.13 $4,651.21

Sheila Cherfilus-McCormick's trip cost:
 )
Traveler $19,590.91 $1,513.23 $1,430.13 $4,651.21
Accompanying /$19,590.91 $1,513.23 $1,430.13 $4,651.21


now downloading information on Lucy McBath's trip to Israel' on Sept. 1, 2023 - Sept. 10, 2023
100% [......................................................] 1359756 / 1359756ReadingDownloads/48Lucy McBathSept. 1, 2023 - Sept. 10, 2023.pdf

now scraping Downloads/48Lucy McBathSept. 1, 2023 - Sept. 10, 2023.pdf
trip cost )
Traveler $17,810.21 $1,692.70 $1,430.13 $4,651.21
Accompanying |$17,526.23 $1,692.70 $1,430.13 $4,651.21

Lucy McBath's trip cost:
 )
Traveler $17,810.21 $1,692.70

now downloading information on Monique Bolsajian's trip to Lithuania' on Aug. 27, 2023 - Sept. 2, 2023
100% [......................................................] 2592767 / 2592767ReadingDownloads/63Monique BolsajianAug. 27, 2023 - Sept. 2, 2023.pdf

now scraping Downloads/63Monique BolsajianAug. 27, 2023 - Sept. 2, 2023.pdf
trip cost )

Traveler $2387.05 $573.48 $570.68 $8.19 Warsaw Uprising
Museum tour

Private transportation
$8.26 Museum of
Accompanying NA N/A N/A Qexeupation and Freedom

Monique Bolsajian's trip cost:
 )

Traveler $2387.05 $573.48 $570.68 $8.19 Warsaw Uprising
Museum tour

Private transportation
$8.26 Museum of
Accompanying NA N/A N/A Qexeupation and Freedom


now downloading information on Sebastian Wigley's trip to Armenia' on Aug. 26, 2023 - Sept. 2, 2023
100% [......................................................] 2126591 / 2126591ReadingDownloads/64Sebastian WigleyAug. 26, 2023 - Sept. 2, 2023.pdf

now scraping Downloads/64Sebastian WigleyAug. 26, 2023 - Se

...

 24% [............                                        ]  2506752 / 10031019

 49% [.........................                           ]  4988928 / 10031019

 76% [.......................................             ]  7692288 / 10031019

100% [....................................................] 10031019 / 10031019ReadingDownloads/469Larry BucshonNov. 9, 2022 - Nov. 14, 2022.pdf



now scraping Downloads/469Larry BucshonNov. 9, 2022 - Nov. 14, 2022.pdf
unable to scrape costs in Downloads/469Larry BucshonNov. 9, 2022 - Nov. 14, 2022.pdf
Larry Bucshon's trip cost:
 Error making sandwich

now downloading information on Linda Sanchez's trip to Spain' on Nov. 9, 2022 - Nov. 14, 2022
100% [......................................................] 2642279 / 2642279ReadingDownloads/470Linda SanchezNov. 9, 2022 - Nov. 14, 2022.pdf

now scraping Downloads/470Linda SanchezNov. 9, 2022 - Nov. 14, 2022.pdf
unable to scrape costs in Downloads/470Linda SanchezNov. 9, 2022 - Nov. 14, 2022.pdf
Linda Sanchez's trip cost:
 Error making sandwich

now downloading information on Randy Feenstra's trip to Spain' on Nov. 9, 2022 - Nov. 14, 2022
100% [......................................................] 1426883 / 1426883ReadingDownloads/471Randy FeenstraNov. 9, 2022 - Nov. 14, 2022.pdf

now scraping Downloads/471Randy FeenstraNov. 9, 2022 - Nov. 14, 2022.pdf
trip cost )
8,544.17 . . $826

In [65]:
df.to_csv("House Travel+Raw Costs.csv")

In [75]:
broken_links

[{'row': 3,
  'traveler': 'Craig Wheeler',
  'Date': 'Dec. 6, 2023 - Dec. 11, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500027304.pdf'},
 {'row': 296,
  'traveler': 'Philip Singleton',
  'Date': 'Aug. 19, 2023 - Aug. 27, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500026994.pdf'},
 {'row': 298,
  'traveler': 'Philip Singleton',
  'Date': 'Aug. 19, 2023 - Aug. 27, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500026994.pdf'},
 {'row': 300,
  'traveler': 'Philip Singleton',
  'Date': 'Aug. 19, 2023 - Aug. 27, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500026994.pdf'},
 {'row': 301,
  'traveler': 'Sally Fox',
  'Date': 'Aug. 19, 2023 - Aug. 27, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500026997.pdf'},
 {'row': 302,
  'traveler': 'Sally Fox',
  'Date': 'Aug. 19, 2023 - Aug. 27, 2023',
  'url': 'https://disclosures-clerk.house.gov/gtimages/ST/2023/500026997.p

In [76]:
no_forms

[[487, 'Zoe Lofgren', 'Aug. 12, 2023 - Aug. 19, 2023', 'Kenya'],
 [1324, 'Ken Calvert', 'March 30, 2023 - April 6, 2023', 'Jordan'],
 [1715, 'Dan Newhouse', 'Nov. 9, 2022 - Nov. 15, 2022', 'Spain'],
 [1726, 'Sarah Jorgenson', 'Nov. 9, 2022 - Nov. 14, 2022', 'Egypt'],
 [1740, 'Gregory Murphy', 'Nov. 9, 2022 - Nov. 14, 2022', 'Egypt'],
 [2239, 'Ann Griffin Johnston', 'Aug. 12, 2022 - Aug. 21, 2022', 'Israel'],
 [2245, 'Tracey Mann', 'Aug. 17, 2022 - Aug. 20, 2022', 'Guatemala'],
 [2617, 'Cristopher Del Beccaro', 'June 25, 2022 - July 3, 2022', 'Israel'],
 [3269,
  'Rosalyn Leighton',
  'April 9, 2022 - April 13, 2022',
  'United Kington'],
 [3290, 'Katharine Durkin', 'March 19, 2022 - March 27, 2022', 'Guatemala'],
 [3291, 'Katharine Durkin', 'March 19, 2022 - March 27, 2022', 'Honduras'],
 [3734, 'Jay Obernolte', 'Oct. 10, 2021 - Oct. 16, 2021', 'Qatar'],
 [3739, 'J. Luis Correa', 'Oct. 10, 2021 - Oct. 16, 2021', 'Qatar'],
 [3757, 'Darin LaHood', 'Oct. 11, 2021 - Oct. 16, 2021', 'Qatar'

In [77]:
broken=pd.DataFrame(broken_links)
broken.to_csv("Broken Links2.csv")

In [78]:
no_forms_df=pd.DataFrame(no_forms)
no_forms_df.to_csv("Form Not Found2.csv")

In [79]:
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\..."
1,1.0,1.0,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich
2,2.0,2.0,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich
3,3.0,3.0,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",NaN
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30046,30046.0,46.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30047,30047.0,47.0,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30048,30048.0,48.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30049,30049.0,49.0,"Sept. 28, 2007 - Sept. 29, 2007",Geoff Davis,Geoff Davis,R,KY-4,"Alburgueque, NM",Home School Legal Defense Association,Original,https://disclosures-clerk.house.gov/gtimages/M...,USA,,NaN


In [98]:
def cleancosts(string):
    reg=r"\$\d+"
    if type(string)==str and "Accompanying" in string:
        traveler, accompanying=string.split("Accompanying")
        traveler=traveler.replace(",","")
        accompanying=accompanying.replace(",","")
        trav_string=re.findall(reg,traveler)
        acc_string=re.findall(reg, accompanying)
        tcosts=[float(x.replace("$","")) for x in trav_string]
        acc_costs=[float(x.replace("$","")) for x in acc_string]
        traveler_costs=sum(tcosts)
        accompanying_costs=sum(acc_costs)
        return [traveler_costs, accompanying_costs]
    else:
        return ["error","error"]
            

In [96]:
#Try that function out
string="apple banana $123.50 $12 $48 Accompanying $530 $650.14"
cleancosts(string)[0]

183.0

In [ ]:
df["Traveler Cost"]=np.nan
df["Accompanying Cost"]=np.nan

In [99]:
df["Traveler Cost"]=df["Raw Cost"].apply(lambda x: cleancosts(x)[0])
df["Accompanying Cost"]=df["Raw Cost"].apply(lambda x: cleancosts(x)[1])

In [100]:
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost,Traveler Cost,Accompanying Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",3440.0,0
1,1.0,1.0,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich,error,error
2,2.0,2.0,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich,error,error
3,3.0,3.0,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",NaN,error,error
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",30078.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30046,30046.0,46.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30047,30047.0,47.0,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30048,30048.0,48.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30049,30049.0,49.0,"Sept. 28, 2007 - Sept. 29, 2007",Geoff Davis,Geoff Davis,R,KY-4,"Alburgueque, NM",Home School Legal Defense Association,Original,https://disclosures-clerk.house.gov/gtimages/M...,USA,,NaN,error,error


In [101]:
df.to_csv("House Travel+Cleaned Costs.csv")

In [102]:
df[df["Traveler Cost"]!="error"]

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost,Traveler Cost,Accompanying Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",3440.0,0
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",30078.0,0
7,7.0,7.0,"Dec. 6, 2023 - Dec. 11, 2023",David Burns,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\nTraveler 1,948.90 $3,050 $1,738* *Averaged ...",")\nTraveler 1,948.90 $3,050 $1,738* *Averaged ...",4788.0,0
11,11.0,11.0,"Dec. 7, 2023 - Dec. 11, 2023",Andrew Garbarino,Andrew Garbarino,R,NY-2,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\nTraveler $13,396.74 $2,222.00 $451.91\n$13,...",")\nTraveler $13,396.74 $2,222.00 $451.91\n$13,...",29123.0,0
13,13.0,13.0,"Dec. 7, 2023 - Dec. 11, 2023",Lori Chavez-DeRemer,Lori Chavez-DeRemer,R,OR-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\ndrayeles $10,913.55 $2,222.00 $451.91\n10,3...",")\ndrayeles $10,913.55 $2,222.00 $451.91\n10,3...",13586.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,7405.0,5.0,"Feb. 15, 2019 - Feb. 22, 2019",Judy Chu,Judy Chu,D,CA-27,"Tel Aviv, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\nPl ittached\n$11,549.69 $717.06 Rlease see...","\n\nPl ittached\n$11,549.69 $717.06 Rlease see...",12266.0,0
7423,7423.0,23.0,"Feb. 15, 2019 - Feb. 22, 2019",Michael F. Doyle,Mike Doyle,D,PA-18,"Jerusalem, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\n$2,080.00 $717.06 Please see attached\n\nd...","\n\n$2,080.00 $717.06 Please see attached\n\nd...",2797.0,0
7424,7424.0,24.0,"Feb. 15, 2019 - Feb. 22, 2019",Michael F. Doyle,Mike Doyle,D,PA-18,"Tel Aviv, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\n$2,080.00 $717.06 Please see attached\n\nd...","\n\n$2,080.00 $717.06 Please see attached\n\nd...",2797.0,0
7436,7436.0,36.0,"Feb. 13, 2019 - Feb. 17, 2019",Meghan Gallagher,Michael McCaul,R,TX-10,"Abuja, Nigeria",International Republican Institute; National D...,Original,https://disclosures-clerk.house.gov/gtimages/S...,Niger Nigeria,"\n\n$189.86 Abuja,\nKaduna\n\nAccompanying\n","\n\n$189.86 Abuja,\nKaduna\n\nAccompanying\n",189.0,0
